In [38]:
import importlib
import data_preprocessing, midi_conversion, model_helpers, models

importlib.reload(data_preprocessing)
importlib.reload(midi_conversion)
importlib.reload(model_helpers)
importlib.reload(models)


<module 'models' from 'c:\\Users\\racer\\OneDrive - Duke University\\CS372\\project_clone\\classical-music-generation-model\\models.py'>

In [8]:
from data_preprocessing import get_midis_by_composer

composers = ["mozart", "haydn"]
midis = get_midis_by_composer(composers)


Now loading MIDIs from data\train.
Could not load data\train\mozart-piano_sonatas-nueva_carpeta-k281_piano_sonata_n03_3mov.mid: Could not decode key with 2 flats and mode 2
Could not load data\train\unknown_artist-i_o-mozart_k550.mid: MThd not found. Probably not a MIDI file
Loaded 311 MIDI files from data\train
Now loading MIDIs from data\val.
Loaded 29 MIDI files from data\val
Now loading MIDIs from data\test.
Could not load data\test\unknown_artist-i_o-mozart_q1_2.mid: MThd not found. Probably not a MIDI file
Loaded 28 MIDI files from data\test
368 MIDI files retrieved.


In [ ]:
from data_preprocessing import process_midis_to_images
import os

train_midis, val_midis, test_midis = midis

# Extract just the MidiFile objects (not composer strings)
train_midi_objs = [m for (m, c) in train_midis]

# Convert all train MIDIs to images
train_images = process_midis_to_images(train_midi_objs)

# Save the images
output_path = "generated/train_images"
os.makedirs(output_path, exist_ok=True)
for i, img in enumerate(train_images):
    img.save(os.path.join(output_path, f"train_window_{i:03d}.png"))

# Repeat process for validation and testing images
val_midi_objs = [m for (m, c) in val_midis]
val_images = process_midis_to_images(val_midi_objs)
output_path = "generated/val_images"
os.makedirs(output_path, exist_ok=True)
for i, img in enumerate(val_images):
    img.save(os.path.join(output_path, f"val_window_{i:03d}.png"))

test_midi_objs = [m for (m, c) in test_midis]
test_images = process_midis_to_images(test_midi_objs)
output_path = "generated/test_images"
os.makedirs(output_path, exist_ok=True)
for i, img in enumerate(test_images):
    img.save(os.path.join(output_path, f"test_window_{i:03d}.png"))

Successfully processed 311 MIDIs into 3605 images.
Successfully processed 29 MIDIs into 345 images.
Successfully processed 28 MIDIs into 336 images.


In [39]:
from data_preprocessing import PianoRollDataset
from torch.utils.data import DataLoader

train_dataset = PianoRollDataset("generated/train_images")
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

Found 3605 images in generated/train_images


In [ ]:
import torch
from models import SimpleUNet, train_diffusion_model

T = 200
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dataset = PianoRollDataset("generated/train_images")
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

model = SimpleUNet().to(device)  # or with smaller channels

losses = train_diffusion_model(model, train_loader, T, num_epochs=200, lr=1e-4, gen_freq=20, device=device)


Found 3605 images in generated/train_images

Generating example every 20 epochs.


Group 1/10:  10%|█         | 2/20 [02:08<19:15, 64.21s/it, loss=0.1013]

In [ ]:
from models import sample_image
from model_helpers import prepare_noise_schedule, show_image_tensor

_, alphas = prepare_noise_schedule(device, timesteps=T)
sample = sample_image(model, alphas, device)     # [1,88,1024]
show_image_tensor(sample, title="Generated piano roll")


In [ ]:
from midi_conversion import pianoroll_images_to_midi
import numpy as np

sample_01 = (sample + 1.0) / 2.0
sample_01 = sample_01.squeeze(0).cpu().numpy()   # [88, 1024] in [0,1]
sample_img = (sample_01 * 255).astype(np.uint8)
img = Image.fromarray(sample_img, mode="L")

mid = pianoroll_images_to_midi([img], tempo_bpm=120, time_signature=(4, 4))
mid.save("generated_diffusion_piece.mid")
